Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
# Imports 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, mean_tweedie_deviance
from sklearn.metrics import explained_variance_score, max_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport
from operator import sub 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Your code starts here:

## **Exploratory Analysis:**

In [3]:
# Load data
TPPI = pd.read_csv('Total_Population_Period_Indicators')
df = TPPI.dropna(subset=['Births'])
# Take a peak
df.head()

,Unnamed: 0,LocID,Location,Time Period,MidPeriod,Total Fertility,NRR(surviving daughters),Crude Birth Rate,Births,Life Expectancy(birth),LExMale,LExFemale,Infant Mortality Rate,Under-five Mortality,Crude Death Rate,Deaths,DeathsMale,DeathsFemale,Net Migration Rate(pK),Net Migrants(K),GrowthRate,Natural Increase Rate,Sex ratio(m per f births,Mean Age Childbearing(f),PopFemale,PopMale,PopTotal,PopDensity
0,0,4,Afghanistan,1950-1955,1953,7.45,1.636,50.314,2015.476,28.61,27.94,29.43,275.866,405.090,36.862,1476.605,794.020,682.585,-0.499,-20.0,1.296,13.452,1.06,29.835,3821.348,4218.336,8039.684,12.315
1,1,4,Afghanistan,1955-1960,1958,7.45,1.765,50.998,2201.589,31.13,30.43,31.97,253.647,374.138,33.718,1455.605,783.110,672.495,-0.463,-20.0,1.683,17.280,1.06,29.835,4176.941,4503.156,8680.097,13.295
2,2,4,Afghanistan,1960-1965,1963,7.45,1.899,51.477,2439.130,33.74,32.99,34.64,230.194,341.957,30.808,1459.784,777.437,682.347,-0.422,-20.0,2.026,20.669,1.06,29.835,4636.170,4907.030,9543.200,14.618
3,3,4,Afghanistan,1965-1970,1968,7.45,2.017,51.646,2728.221,36.15,35.37,37.06,211.072,314.708,28.223,1490.896,790.745,700.151,-0.379,-20.0,2.307,23.423,1.06,29.835,5202.606,5434.458,10637.064,16.293
4,4,4,Afghanistan,1970-1975,1973,7.45,2.141,51.234,3056.451,38.74,37.92,39.67,191.642,286.630,25.495,1520.938,802.414,718.524,-0.335,-20.0,2.544,25.739,1.06,29.835,5951.120,6157.843,12108.963,18.548


In [4]:
# Explore Data
print(df.info())
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7088 entries, 0 to 7631
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                7088 non-null   int64  
 1   LocID                     7088 non-null   int64  
 2   Location                  7088 non-null   object 
 3   Time Period               7088 non-null   object 
 4   MidPeriod                 7088 non-null   int64  
 5   Total Fertility           7088 non-null   float64
 6   NRR(surviving daughters)  7088 non-null   float64
 7   Crude Birth Rate          7088 non-null   float64
 8   Births                    7088 non-null   float64
 9   Life Expectancy(birth)    7088 non-null   float64
 10  LExMale                   7088 non-null   float64
 11  LExFemale                 7088 non-null   float64
 12  Infant Mortality Rate     7088 non-null   float64
 13  Under-five Mortality      7088 non-null   float64
 14  Crude De

,Unnamed: 0,LocID,MidPeriod,Total Fertility,NRR(surviving daughters),Crude Birth Rate,Births,Life Expectancy(birth),LExMale,LExFemale,Infant Mortality Rate,Under-five Mortality,Crude Death Rate,Deaths,DeathsMale,DeathsFemale,Net Migration Rate(pK),Net Migrants(K),GrowthRate,Natural Increase Rate,Sex ratio(m per f births,Mean Age Childbearing(f),PopFemale,PopMale,PopTotal,PopDensity
count,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7.088000e+03,7.088000e+03,7.088000e+03,7088.000000
mean,3814.994357,1077.688488,1994.875000,3.831392,1.534372,27.639051,57279.246641,64.279876,62.067798,66.536494,59.517827,87.951534,11.360619,24977.727445,13181.393661,11796.333783,-0.130939,-214.672219,1.617724,16.278433,1.053711,28.810993,2.452477e+05,2.489027e+05,4.941504e+05,141.007502
std,2210.800671,729.124394,33.862164,1.897975,0.604770,12.934401,119362.543037,12.607073,12.158732,13.112974,52.344031,85.371346,5.500324,53517.256616,28209.979282,25322.540725,7.185746,2824.850320,1.247379,10.526515,0.017371,1.415274,5.437541e+05,5.560014e+05,1.099675e+06,827.146104
min,0.000000,4.000000,1953.000000,0.850000,0.410000,5.558000,1.198000,14.490000,11.880000,18.120000,0.126000,0.231000,1.147000,1.597000,0.808000,0.681000,-70.787000,-24462.517000,-5.321000,-20.979000,1.004000,23.897000,7.923000e+00,9.321000e+00,1.724400e+01,0.070000
25%,1915.750000,462.000000,1971.750000,2.079750,0.970000,15.911750,602.718250,55.340000,53.580000,57.050000,16.031250,19.552750,7.508500,287.015000,150.790000,136.603250,-1.400000,-457.890000,0.746000,7.604750,1.050000,27.712000,2.499810e+03,2.478078e+03,4.949038e+03,17.057500
50%,3783.500000,922.000000,1990.500000,3.301000,1.457500,25.893000,5639.506000,66.565000,64.045000,69.240000,43.623500,56.928500,9.880500,2716.019000,1410.282000,1289.021000,-0.239000,-17.073000,1.663000,17.738000,1.050000,28.997000,2.588380e+04,2.529137e+04,5.135256e+04,37.360000
75%,5779.250000,1596.000000,2009.250000,5.642000,2.039000,39.644750,51853.495500,73.340000,70.500000,76.290000,93.709250,137.602000,13.356750,20751.778250,11056.459000,9867.950000,0.859250,88.075000,2.431000,25.183500,1.060000,29.703250,2.122962e+05,2.065572e+05,4.184542e+05,102.086000
max,7631.000000,5501.000000,2098.000000,8.800000,3.653000,58.263000,701277.931000,94.020000,91.140000,96.930000,319.239000,465.517000,61.634000,606048.782000,313187.571000,292861.211000,134.414000,23278.467000,16.986000,42.294000,1.173000,34.997000,5.425119e+06,5.443229e+06,1.086835e+07,33470.669000


In [5]:
# Get Pandas Profiling Report
profile = ProfileReport(df, minimal=True).to_notebook_iframe()
profile


In [6]:
# Frequency of values in the 'Births' column
df.Births.value_counts()
# The specificity of the values, being that they are births is questionable
# Indicative of mean values of grouped data over periods of time (reasonable given the dataframe is based on period data)
# for combined groups

184439.615    5
23638.907     5
21438.111     5
128009.532    5
102931.137    5
             ..
342.520       1
1310.475      1
2133.004      1
39760.368     1
5.750         1
Name: Births, Length: 6535, dtype: int64

In [7]:
# Looking further into it
top5   = df.Births.value_counts().head(5).index
# pulling up the rows with those similar numbers. 
df.loc[df.Births.isin(top5)]
# I feel like the locations that group data from different locations won't be problematic for answering my questions
# I do want to make note that there are duplicated rows

,Unnamed: 0,LocID,Location,Time Period,MidPeriod,Total Fertility,NRR(surviving daughters),Crude Birth Rate,Births,Life Expectancy(birth),LExMale,LExFemale,Infant Mortality Rate,Under-five Mortality,Crude Death Rate,Deaths,DeathsMale,DeathsFemale,Net Migration Rate(pK),Net Migrants(K),GrowthRate,Natural Increase Rate,Sex ratio(m per f births,Mean Age Childbearing(f),PopFemale,PopMale,PopTotal,PopDensity
1537,1537,2098,Countries with Access to the Sea: Northern Ame...,1955-1960,1958,3.608,1.689,24.116,23638.907,69.70,66.66,73.02,27.662,32.025,9.293,9108.861,5184.542,3924.319,2.743,2688.711,1.758,14.823,1.05,26.592,99564.829,98305.348,197870.177,10.609
1549,1549,2098,Countries with Access to the Sea: Northern Ame...,2015-2020,2018,1.753,0.843,11.813,21438.111,79.15,76.69,81.63,5.700,6.687,8.586,15581.517,8023.084,7558.433,3.296,5982.000,0.652,3.227,1.05,29.416,183994.075,180301.919,364295.994,19.532
2098,2098,1056,ESCAP: SAARC,1960-1965,1963,6.037,1.888,42.294,128009.532,43.46,44.16,42.73,157.747,234.320,20.674,62571.763,31486.465,31085.298,-0.150,-454.118,2.149,21.620,1.05,29.439,294263.082,316235.226,610498.308,127.945
2103,2103,1056,ESCAP: SAARC,1985-1990,1988,4.520,1.748,34.371,184439.615,56.82,56.55,57.14,96.688,137.304,11.549,61972.109,32324.875,29647.234,-0.357,-1914.631,2.249,22.822,1.08,27.665,522658.804,561622.355,1084281.159,227.237
2111,2111,1056,ESCAP: SAARC,2095-2100,2098,1.737,0.823,9.620,102931.137,81.20,79.80,82.68,6.806,8.319,13.423,143613.427,73688.621,69924.806,-0.547,-5854.611,-0.435,-3.803,1.08,30.503,1054923.694,1080554.587,2135478.281,447.541
4625,4625,1485,More developed: Northern America,1955-1960,1958,3.608,1.689,24.116,23638.907,69.70,66.66,73.02,27.662,32.025,9.293,9108.861,5184.542,3924.319,2.743,2688.711,1.758,14.823,1.05,26.592,99564.829,98305.348,197870.177,10.609
4637,4637,1485,More developed: Northern America,2015-2020,2018,1.753,0.843,11.813,21438.111,79.15,76.69,81.63,5.700,6.687,8.586,15581.517,8023.084,7558.433,3.296,5982.000,0.652,3.227,1.05,29.416,183994.075,180301.919,364295.994,19.532
5026,5026,905,Northern America,1955-1960,1958,3.608,1.689,24.116,23638.907,69.70,66.66,73.02,27.662,32.025,9.293,9108.861,5184.542,3924.319,2.743,2688.711,1.758,14.823,1.05,26.592,99564.829,98305.348,197870.177,10.609
5027,5027,918,Northern America,1955-1960,1958,3.608,1.689,24.116,23638.907,69.70,66.66,73.02,27.662,32.025,9.293,9108.861,5184.542,3924.319,2.743,2688.711,1.758,14.823,1.05,26.592,99564.829,98305.348,197870.177,10.609
5050,5050,918,Northern America,2015-2020,2018,1.753,0.843,11.813,21438.111,79.15,76.69,81.63,5.700,6.687,8.586,15581.517,8023.084,7558.433,3.296,5982.000,0.652,3.227,1.05,29.416,183994.075,180301.919,364295.994,19.532


In [8]:
# baseline regression model (from previous assignment)
baseline = round(df.Births.mean())
baselist = [baseline] * len(df.Births)
errs = baseline - df['Births']
mae = errs.abs().mean()
print(f'The baseline is {baseline} births,')
print(f'On average, that would be off by {mae:,.0f}.')

The baseline is 57279 births,
On average, that would be off by 72,921.


In [9]:
# I chose to go with a Regression Metrics function I had previously used in other assignments
def RM(data, pred):
  ''' Upon receiving the true data and predicted data, prints various 
  regression metrics'''

  # average squared difference 
  mse = mean_squared_error(y_true=data,y_pred= pred)
  # error of a model
  rmse = np.sqrt(mse)
  # measure of errors between observations 
  mae = mean_absolute_error(data,pred)
  # measure of how close the data is to the fitted regression line
  R2 = r2_score(data,pred)
  # R2 score adjusted to predictors 
  AdR2 = float(1-(1-R2)*(len(data)- 1)/(len(data)- data.nunique() - 1))
  # generalization of least squared error to non Gaussian error distributions
  meanPD = mean_poisson_deviance(data, pred)
  meanGD = mean_gamma_deviance(data,pred)
  meanTD = mean_tweedie_deviance(data,pred)
  # measure the discrepancy between a model and actual data
  evs = explained_variance_score(data,pred)
  # maximum difference between the point estimate and the actual parameter
  me = max_error(data,pred)

  print(f'Mean Squared Error: {mse:.2f}')
  print(f'Root Mean Squared Error: {rmse:.2f}')
  print(f'Mean Absolute Error: {mae:.2f}')
  print(f'R^2: {R2}')
  print(f'Adjusted R^2: {AdR2}')
  print(f'Explained Variance Score: {evs}')
  print(f'Max Error:{me}')
  print(f'Mean Tweedie Deviance: {meanTD}')
  print(f'Mean Poisson Deviance: {meanPD}')
  print(f'Mean Gamma Deviance: {meanGD}')
# The results were:
RM(df.Births,baselist)

Mean Squared Error: 14245406604.59
Root Mean Squared Error: 119354.12
Mean Absolute Error: 72920.87
R^2: -4.270139797313277e-12
Adjusted R^2: -11.838768115996853
Explained Variance Score: 0.0
Max Error:643998.931
Mean Tweedie Deviance: 14245406604.59393
Mean Poisson Deviance: 145662.68013098973
Mean Gamma Deviance: 4.957275389216257


In [10]:
# Train/Val/Test split
train,Test = train_test_split(df, train_size=0.80, test_size=0.20)
Train, Val = train_test_split(train,train_size=0.80, test_size=0.20)
Train.shape,Val.shape,Test.shape

((4536, 28), (1134, 28), (1418, 28))

## **Fast First Model:**

Attempt to make a fast model that beats guessing with the baseline. I decided to see how much an improvement it would be with a linear regression model.

In [11]:
target = ['Births']
feature = ['PopFemale']

In [12]:
px.scatter(df,x='Births',y='PopFemale', trendline='ols')

In [13]:
# instantiate class

model = LinearRegression()

# arrange x and y

x_train = Train[feature]
x_val   = Val[feature]
x_test  = Test[feature]

y_train = Train[target]
y_val   = Val[target]

# fit the model

model.fit(x_train,y_train)

# apply to new data

val_pred = model.predict(x_val)


In [14]:
# the Regression metrics from the quick model
RM(y_val,val_pred)

Mean Squared Error: 1817065182.65
Root Mean Squared Error: 42627.05
Mean Absolute Error: 21852.66
R^2: 0.8564560228895101
Adjusted R^2: -17.07059178513168
Explained Variance Score: 0.856470939423808
Max Error:464667.7406709804
Mean Tweedie Deviance: 1817065182.6467328
Mean Poisson Deviance: 15121.115588098344
Mean Gamma Deviance: 2.361145577724485


### How does the two compare to each other?

In [15]:
# Create function that compares results from the regression metrics function

def RMComparison(data, pred, data2, pred2):

  ''' Upon receiving the x and y values of two models 
  returns results of the differences between the two'''

  # mean square error
  mse = mean_squared_error(data,pred) - mean_squared_error(data2,pred2)
  # root mean square error
  rmse = np.sqrt(mean_squared_error(data,pred)) - np.sqrt(mean_squared_error(data2,pred2))
  # mean absolute error
  mae = mean_absolute_error(data,pred) - mean_absolute_error(data2,pred2)
  # R2 score
  R2 = r2_score(data,pred) - r2_score(data2,pred2)
  # Adjusted R2 score
  AdR2 = float(1-(1-(r2_score(data,pred)))*(len(data)- 1)/(len(data)- data.nunique() - 1)) - float(1-(1-(r2_score(data2,pred2)))*(len(data2)- 1)/(len(data2)- data.nunique() - 1))
  # Tweedie deviances
  MPD = mean_poisson_deviance(data, pred) - mean_poisson_deviance(data2, pred2)
  MGD = mean_gamma_deviance(data,pred) - mean_gamma_deviance(data2,pred2)
  MTD = mean_tweedie_deviance(data,pred) - mean_tweedie_deviance(data2,pred2)
  # Explained Variance score
  EVS = explained_variance_score(data,pred) - explained_variance_score(data2,pred2)
  # max error
  ME  = max_error(data,pred) - max_error(data2,pred2)

  print(f'Mean Squared Error Difference: {mse:.2f}')
  print(f'Root Mean Squared Error Difference: {rmse:.2f}')
  print(f'Mean Absolute Error Difference: {mae:.2f}')
  print(f'R^2 Score Difference: {R2}')
  print(f'Adjusted R^2 Difference: {AdR2}')
  print(f'Explained Variance Score Difference: {EVS}')
  print(f'Max Error Difference:{ME}')
  print(f'Mean Tweedie Deviance Difference: {MTD}')
  print(f'Mean Poisson Deviance Difference: {MPD}')
  print(f'Mean Gamma Deviance Difference: {MGD}')

In [16]:
print('There is a definite improvement between the models')
RMComparison(df.Births, baselist,y_val,val_pred)

There is a definite improvement between the models
Mean Squared Error Difference: 12428341421.95
Root Mean Squared Error Difference: 76727.08
Mean Absolute Error Difference: 51068.21
R^2 Score Difference: -0.8564560228937802
Adjusted R^2 Difference: -12.86887461841562
Explained Variance Score Difference: -0.856470939423808
Max Error Difference:179331.19032901956
Mean Tweedie Deviance Difference: 12428341421.947197
Mean Poisson Deviance Difference: 130541.56454289138
Mean Gamma Deviance Difference: 2.5961298114917715
